<h1>Notebook feta per Rubén Álvarez i Pol Pérez </h1>

In [32]:
%pip install -r requirements.txt --ignore-installed

  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached scikit_learn-1.4.1.post1-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached regex-2023.12.25-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.0 kB 320.0 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 643.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 1.6 MB/s eta 0:00:00
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached zope.event-5.0-py3-none-any.whl.metadata (4.4 kB)
     ---------------------------------------- 0.0/43.1 kB ? eta -:--:--
     -

ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\ralva\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tornado\\speedups.pyd'
Check the permissions.


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ralva\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


<h1> <a href= https://gebakx.github.io/plh/s4/index.html#14>Pràctica 2.a: Detecció d'opinions (Supervised Learning) </a> </h1>

___

## Importacions i lectura del text inicial

In [ ]:
# Lectura de dades (Movie Reviews Corpus) NLTK

import nltk
import re
import sklearn
from sklearn.model_selection import train_test_split 
nltk.download('movie_reviews')
nltk.download('wordnet')
from nltk.corpus import movie_reviews as mr
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


In [3]:
all_texts = []
all_categories = []

for fileid in mr.fileids():
    all_texts.append(mr.raw(fileid))
    all_categories.append(mr.categories(fileid)[0])

In [4]:
all_categories.count('neg')

1000

## Preprocessat

In [5]:
def preprocessing(text : str) -> str:
  """Preprocessat de text per a anàlisi de sentiment. Aplica:
      - Eliminació de dígits
      - Conversió a minúscules
      - Substitució de dos espais per un
      - Eliminació de tabs
      - Lematització

  Args:
      text (str): text to preprocess

  Returns:
      String: text preprocessed
  """

  for num in range(10):
    text = text.replace(str(num), "")

  text = text.lower()

  text = re.sub(" +", " ", text)

  text = text.replace("\n ", "\n  ")

  text = text.replace('\t','')

  lemmatizer = WordNetLemmatizer()
  text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
  
  return text

In [6]:
#Apliquem el preprocès a tots els textos

for text in all_texts:
  text = preprocessing(text)

## Definició i validació del model

### Separació train i test


In [7]:
# Separem les dades en train i test
X_train, X_test, y_train, y_test = train_test_split(all_texts, all_categories, test_size=0.2, random_state=14)

### Aplicació Count Vectorizer

In [8]:
# Crear un objecte CountVectorizer eliminant les stopwords
vectorizer = CountVectorizer(stop_words=stopwords.words('english'), min_df=5)

# Ajustar el vectoritzador a les dades d'entrenament
train_matrix = vectorizer.fit_transform(X_train)

# Transformar les dades de test
test_matrix = vectorizer.transform(X_test)

### Grid search cross validation (Random Forest)

In [9]:
#Hyperparàmetres
n_estimators = [100, 200, 500]
max_depth = [3, 5, 10]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2]

In [10]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}

rf = RandomForestClassifier()
rf_cv = GridSearchCV(rf, param_grid, cv=5)
rf_cv.fit(train_matrix, y_train)
print("Tuned Random Forest Parameters: {}".format(rf_cv.best_params_))

Tuned Random Forest Parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 500}


### Random Forest amb els millors paràmetres

In [11]:
# Random forest
from sklearn.ensemble import RandomForestClassifier

# Crear un objecte RandomForestClassifier per saber si el text és positiu o negatiu
classifier = RandomForestClassifier(random_state=14, n_estimators=rf_cv.best_params_['n_estimators'], max_depth=rf_cv.best_params_['max_depth'], min_samples_split=rf_cv.best_params_['min_samples_split'], min_samples_leaf=rf_cv.best_params_['min_samples_leaf'])

# Entrenar el classificador
classifier.fit(train_matrix, y_train)

# Predir las categories de les dades de validació
test_predictions = classifier.predict(test_matrix)

## Resultats (aplicant el model a les dades test)

In [23]:
for i, prediction in enumerate(test_predictions):
    print(f"Prediction: {prediction}")
    print(f"Real_category: {y_test[i]}")
    print("-----------------------------")
    print(f"Text: {X_test[i][:100]}")

Prediction: neg
Real_category: pos
-----------------------------
Text: when you go to the movies as much as i do , you unfortunately end up seeing certain movie trailers o
Prediction: pos
Real_category: pos
-----------------------------
Text: hollywood has really done the whole dracula thing to death ( no pun intended - honest ) . 
it's a sh
Prediction: neg
Real_category: neg
-----------------------------
Text: i never understood what the clich ? " hell on earth " truly meant until very recently . 
i've just n
Prediction: neg
Real_category: pos
-----------------------------
Text: whenever studio executives try to combine the successful formula of two blockbuster films , the end 
Prediction: neg
Real_category: neg
-----------------------------
Text:  " return to horror high , " wants to be a couple different types of movies at once . 
the film tell
Prediction: pos
Real_category: neg
-----------------------------
Text: don't get me wrong -- i tend to appreciate besson's naive , unashamed

In [13]:
# Calcular la precisión
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, test_predictions)

accuracy

0.8425

## Predicció per un text importat de fora

In [14]:
def predict_sentiment(text):
  """Predicts the sentiment of a text

  Args:
      text (str): text to predict

  Returns:
      String: sentiment prediction
  """
  
  text = preprocessing(text)
  text_matrix = vectorizer.transform([text])
  prediction = classifier.predict(text_matrix)
  return prediction[0]

In [15]:
predict_sentiment("""A world, by the way, systemic, where the physics and artificial intelligence of the characters and enemies generate a lot of spontaneous moments. Especially in the combats, constant and hilarious, especially when giant monsters are involved; and very customizable thanks to the great vocation and pawn systems. We must point out the performance flaws, a somewhat limited bestiary and a visual section with its lights and shadows, but in the end they are minor details in a title that has made us say out loud several times "I can't believe it." Such a surprising title that teaches you to expect the unexpected.""")

'pos'

<h1> <a href= https://gebakx.github.io/plh/s4/index.html#17> Pràctica 2.b: Detecció d'opinions (Unsupervised Learning) </a> </h1>

___

## Importacions i lectura del text inicial

In [17]:
import nltk
import re
import sklearn
from sklearn.model_selection import train_test_split 
nltk.download('movie_reviews')
nltk.download('wordnet')
from nltk.corpus import movie_reviews as mr
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\ralva\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ralva\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
all_texts = []
all_categories = []

for fileid in mr.fileids():
    all_texts.append(mr.raw(fileid))
    all_categories.append(mr.categories(fileid)[0])

## Preprocessat

In [39]:
def preprocessing(text):
  """Preprocessat de text per a anàlisi de sentiment. Aplica:
      - Eliminació de dígits
      - Conversió a minúscules
      - Substitució de dos espais per un
      - Eliminació de tabs
      - Lematització

  Args:
      text (str): text to preprocess

  Returns:
      String: text preprocessed
  """
  
  for num in range(10):
    text = text.replace(str(num), "")

  text = text.lower()

  text = re.sub(" +", " ", text)

  text = text.replace("\n ", "\n  ")

  text = text.replace('\t','')
  
  lemmatizer = WordNetLemmatizer()
  text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
  
  # Tokenitzar el text
  text = nltk.word_tokenize(text)
  
  return text

In [40]:
for text in all_texts:
  text = preprocessing(text)

In [41]:
all_texts[0]

'plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat\'s the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn\'t snag this one correctly . \nthey seem to have taken this pretty neat concept , but executed it terribly . \nso what are the problems with the movie ? \nwell , its main problem is that it\'s simply too jumbled . \nit starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience membe

## Tokenització

In [ ]:
# Tokenizar los datos
